# QPQ9 Sample

In [1]:
%matplotlib inline

In [2]:
# imports
from scipy.io import readsav as scirs
import copy, os, glob, imp
import numpy as np

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Column, Table
from astropy.io import fits, ascii

from xastropy.xutils import fits as xxf
from xastropy.xutils import xdebug as xdb

from specdb.specdb import IgmSpec, SpecDB
from specdb import query_catalog as spqcat
from specdb import interface_group as spigp
from specdb import utils as spdbu

from linetools import utils as ltu

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.26)

import enigma

## QPQ Sample Code
Modifying lya_qpq_sample.pro in pro/data/photo

Created QPQ-CGM-IR  [Joe did]

QPQ-CGM-IR_00.000-00.300_Sat-Apr-25-21:33:45-2015.sav

### Try MgII $z_{\rm em}$ someday

## Read the Sample and Collate

In [3]:
'''
# IDL savefile read failed..
#dat = scirs(os.getenv('DROPBOX_DIR')+'/QSOPairs/QSO-CGM-IR/QPQ-CGM-IR_00.000-00.300_Sat-Apr-25-21:33:45-2015.sav',verbose=True)
#dat

# Wrote a binary FITS table from IDL
#bgfil = 'QPQ-CGM-IR_00.000-00.300_Sat-Apr-25-21:33:45-2015_qsobg.fits.gz'
#fgfil = 'QPQ-CGM-IR_00.000-00.300_Sat-Apr-25-21:33:45-2015_qsofg.fits.gz'
#
bgfil = 'QPQ-CGM-IR_00.000-00.300_Tue-May-26-18:12:57-2015_qsobg.fits'
fgfil = 'QPQ-CGM-IR_00.000-00.300_Tue-May-26-18:12:57-2015_qsofg.fits'
#
qso_bg = xxf.bintab_to_table(os.getenv('DROPBOX_DIR')+'/QSOPairs/QPQ-CGM-IR/'+bgfil)
#
qso_fg = xxf.bintab_to_table(os.getenv('DROPBOX_DIR')+'/QSOPairs/QPQ-CGM-IR/'+fgfil)
#qso_fg[['NAME','LYA_FILE', 'Z_IR']]
'''

"\n# IDL savefile read failed..\n#dat = scirs(os.getenv('DROPBOX_DIR')+'/QSOPairs/QSO-CGM-IR/QPQ-CGM-IR_00.000-00.300_Sat-Apr-25-21:33:45-2015.sav',verbose=True)\n#dat\n\n# Wrote a binary FITS table from IDL\n#bgfil = 'QPQ-CGM-IR_00.000-00.300_Sat-Apr-25-21:33:45-2015_qsobg.fits.gz'\n#fgfil = 'QPQ-CGM-IR_00.000-00.300_Sat-Apr-25-21:33:45-2015_qsofg.fits.gz'\n#\nbgfil = 'QPQ-CGM-IR_00.000-00.300_Tue-May-26-18:12:57-2015_qsobg.fits'\nfgfil = 'QPQ-CGM-IR_00.000-00.300_Tue-May-26-18:12:57-2015_qsofg.fits'\n#\nqso_bg = xxf.bintab_to_table(os.getenv('DROPBOX_DIR')+'/QSOPairs/QPQ-CGM-IR/'+bgfil)\n#\nqso_fg = xxf.bintab_to_table(os.getenv('DROPBOX_DIR')+'/QSOPairs/QPQ-CGM-IR/'+fgfil)\n#qso_fg[['NAME','LYA_FILE', 'Z_IR']]\n"

In [4]:
'''
# Collate Tables, calculate R_perp
qpq9 = copy.deepcopy(qso_bg)
# Add/replce key columns from qso_fg
oldc = ['Z_IR','SIGMA_ZIR', 'RA', 'DEC']
newc = ['FG_ZIR', 'FG_SIG_ZIR', 'FG_RA', 'FG_DEC']
for jj,ioldc in enumerate(oldc):
    tmpc = Column( qso_fg[ioldc], name=newc[jj])
    #print newc[jj]
    qpq9.add_column( tmpc )
# 
qpq9['G_UV'] = qso_fg['G_UV']
qpq9.add_column( Column( qso_bg['Z_TOT'], name='BG_Z') )
len(qpq9)
'''

"\n# Collate Tables, calculate R_perp\nqpq9 = copy.deepcopy(qso_bg)\n# Add/replce key columns from qso_fg\noldc = ['Z_IR','SIGMA_ZIR', 'RA', 'DEC']\nnewc = ['FG_ZIR', 'FG_SIG_ZIR', 'FG_RA', 'FG_DEC']\nfor jj,ioldc in enumerate(oldc):\n    tmpc = Column( qso_fg[ioldc], name=newc[jj])\n    #print newc[jj]\n    qpq9.add_column( tmpc )\n# \nqpq9['G_UV'] = qso_fg['G_UV']\nqpq9.add_column( Column( qso_bg['Z_TOT'], name='BG_Z') )\nlen(qpq9)\n"

### Rphys

In [5]:
'''
# Coordinates
c_fg = SkyCoord(ra=qpq9['FG_RA']*u.deg, dec=qpq9['FG_DEC']*u.deg)
c_bg = SkyCoord(ra=qpq9['RA']*u.deg, dec=qpq9['DEC']*u.deg)
# Separation
kpc_amin = cosmo.kpc_proper_per_arcmin( qpq9['FG_ZIR'] ) # kpc per arcmin
ang_sep = c_fg.separation(c_bg).to('arcmin')
Rphys = kpc_amin * ang_sep
# Add
if 'R_PHYS' not in qpq9.dtype.names:
    qpq9.add_column(Column( Rphys, name='R_PHYS'))
len(qpq9)
'''

"\n# Coordinates\nc_fg = SkyCoord(ra=qpq9['FG_RA']*u.deg, dec=qpq9['FG_DEC']*u.deg)\nc_bg = SkyCoord(ra=qpq9['RA']*u.deg, dec=qpq9['DEC']*u.deg)\n# Separation\nkpc_amin = cosmo.kpc_proper_per_arcmin( qpq9['FG_ZIR'] ) # kpc per arcmin\nang_sep = c_fg.separation(c_bg).to('arcmin')\nRphys = kpc_amin * ang_sep\n# Add\nif 'R_PHYS' not in qpq9.dtype.names:\n    qpq9.add_column(Column( Rphys, name='R_PHYS'))\nlen(qpq9)\n"

## Cuts

In [6]:
'''
# Lya beta
zLya_beta = (qpq9['BG_Z']+1)*1025.7223 / 1215.6701 - 1
dv_beta = ltu.v_from_z(qpq9['FG_ZIR'], zLya_beta)
gd_beta = dv_beta > (-500*u.km/u.s)
#
qpq9 = qpq9[gd_beta]
print('We have {:d} pairs in QPQ9'.format(len(qpq9)))
#xdb.xpcol(zLya_beta, qpq9['FG_ZIR'], qpq9['BG_Z'])
'''

"\n# Lya beta\nzLya_beta = (qpq9['BG_Z']+1)*1025.7223 / 1215.6701 - 1\ndv_beta = ltu.v_from_z(qpq9['FG_ZIR'], zLya_beta)\ngd_beta = dv_beta > (-500*u.km/u.s)\n#\nqpq9 = qpq9[gd_beta]\nprint('We have {:d} pairs in QPQ9'.format(len(qpq9)))\n#xdb.xpcol(zLya_beta, qpq9['FG_ZIR'], qpq9['BG_Z'])\n"

## Write

In [7]:
# Don't write. The current qpq9_zIR.fits made by Joe/X have more columns then what the code above gives. 
#xxf.table_to_fits(qpq9, 'qpq9_zIR.fits')

# list of NIR redshifts 

In [9]:
# Load masterlists
path = enigma.__path__[0]+'/data/redshifts/'
c_NIR = []
gnirs = Table.read(path+'gnirs_masterlist.dat',guess=False,format='ascii.csv',comment='#')
for qq in gnirs:
    c_NIR.append(ltu.radec_to_coord(qq['ra_qso']+' '+qq['dec_qso']))
niri = Table.read(path+'niri_masterlist.dat',guess=False,format='ascii.csv',comment='#')
for qq in niri:
    c_NIR.append(ltu.radec_to_coord(qq['ra_qso']+' '+qq['dec_qso']))
nirspec = Table.read(path+'nirspec_masterlist.dat',guess=False,format='ascii.csv',comment='#')
for qq in nirspec:
    c_NIR.append(ltu.radec_to_coord(qq['ra_qso']+' '+qq['dec_qso']))
xshooter = Table.read(path+'xshooter_masterlist.dat',guess=False,format='ascii.csv',comment='#')
for qq in xshooter:
    c_NIR.append(ltu.radec_to_coord(qq['ra_qso']+' '+qq['dec_qso']))
tspec = Table.read(path+'triplespec_masterlist.dat',guess=False,format='ascii.csv',comment='#')
for qq in tspec:
    c_NIR.append(ltu.radec_to_coord(qq['ra_qso']+' '+qq['dec_qso']))
isaac = Table.read(path+'isaac_masterlist.dat',guess=False,format='ascii.csv',comment='#')
for qq in isaac:
    c_NIR.append(ltu.radec_to_coord(qq['ra_qso']+' '+qq['dec_qso']))
lucifer = Table.read(path+'lucifer_masterlist.dat',guess=False,format='ascii.csv',comment='#')
for qq in lucifer:
    c_NIR.append(ltu.radec_to_coord(qq['ra_qso']+' '+qq['dec_qso']))    
liam = Table.read(path+'masterlist_liam.fits')
for qq in liam:
    c_NIR.append(SkyCoord(qq['RAJ2000'],qq['DEJ2000']))
c_NIR = SkyCoord(c_NIR)

# Load zem structures
zsys_line = ['H$\\alpha$','H$\\beta$','MgII','[OIII]']
zem_list = []
zem_coord = []
zem_fil = glob.glob(os.getenv('QPQ9')+'Analysis/Redshifts/zem/From_X_Joe/*')
for ff in zem_fil:
    zem = xxf.bintab_to_table(ff)
    zem_list.append(zem)
    ipos = zem['SPEC_FIL'][0].rfind('/SDSSJ')+6
    radec = zem['SPEC_FIL'][0][ipos:]
    ipos2 = radec.rfind('+')
    if ipos2 == -1:
        ipos2 = radec.rfind('-')
    ipos3 = radec.rfind('_F.fits')
    if ipos3 == -1:
        ipos3 = radec.rfind('.fits')
    RA = radec[0:2]+'h'+radec[2:4]+'m'+radec[4:ipos2]+'s'
    dec = radec[ipos2:ipos2+3]+'d'+radec[ipos2+3:ipos2+5]+'m'+radec[ipos2+5:ipos3]+'s'
    zem_coord.append(SkyCoord(RA,dec))
zem_fil = glob.glob(os.getenv('QPQ9')+'Analysis/Redshifts/zem/*_zem.fits')
for ff in zem_fil:
    zem = xxf.bintab_to_table(ff)
    zem_list.append(zem)
    ipos = zem['SPEC_FIL'][0].rfind('/SDSSJ')+6
    radec = zem['SPEC_FIL'][0][ipos:]
    RA = radec[0:2]+'h'+radec[2:4]+'m'+radec[4:9]+'s'
    dec = radec[9:12]+'d'+radec[12:14]+'m'+radec[14:18]+'s'
    zem_coord.append(SkyCoord(RA,dec))
zem_coord = SkyCoord(zem_coord)

x.fits.bintab_to_table: Reading /Users/lwymarie/Documents/papers/QPQ9/Analysis/Redshifts/zem/From_X_Joe/SDSSJ005717.36-000113.3_zem.fits
x.fits.bintab_to_table: Reading /Users/lwymarie/Documents/papers/QPQ9/Analysis/Redshifts/zem/From_X_Joe/SDSSJ022517.68+004821.9_zem.fits
x.fits.bintab_to_table: Reading /Users/lwymarie/Documents/papers/QPQ9/Analysis/Redshifts/zem/From_X_Joe/SDSSJ080049.89+354249.6_zem.fits
x.fits.bintab_to_table: Reading /Users/lwymarie/Documents/papers/QPQ9/Analysis/Redshifts/zem/From_X_Joe/SDSSJ083757.91+383727.1_zem.fits
x.fits.bintab_to_table: Reading /Users/lwymarie/Documents/papers/QPQ9/Analysis/Redshifts/zem/From_X_Joe/SDSSJ084158.47+392121.0_zem.fits
x.fits.bintab_to_table: Reading /Users/lwymarie/Documents/papers/QPQ9/Analysis/Redshifts/zem/From_X_Joe/SDSSJ085358.36-001108.0_zem.fits
x.fits.bintab_to_table: Reading /Users/lwymarie/Documents/papers/QPQ9/Analysis/Redshifts/zem/From_X_Joe/SDSSJ091046.40+041458.5_zem.fits
x.fits.bintab_to_table: Reading /Users/lw

# Make a new sample merging QPQ9_zIR and QPQ7

In [10]:
zIR_fil = 'qpq9_zIR.fits'
QPQ7_fil = imp.find_module('enigma')[1] + '/data/qpq/qpq7_pairs.fits.gz'
QPQ_zIR = xxf.bintab_to_table(zIR_fil)
QPQ7 = xxf.bintab_to_table(QPQ7_fil)

x.fits.bintab_to_table: Reading qpq9_zIR.fits
x.fits.bintab_to_table: Reading /Users/lwymarie/python/enigma/enigma/data/qpq/qpq7_pairs.fits.gz


In [11]:
QPQ9 = copy.deepcopy(QPQ_zIR)
QPQ9.rename_column('FG_ZIR', 'Z_FG')
QPQ9.rename_column('FG_SIG_ZIR','Z_FSIG')
for ii,qq in enumerate(QPQ9):
    try:
        assert 'J0225+0048' in qq['NAME'] # redshift is marked to be OII by mistake
        QPQ9.remove_row(ii)
    except AssertionError:
        pass
    try:
        assert 'J0913-0107' in qq['NAME'] # Hb redshift is not good, use MgII
        for zz,zem in enumerate(zem_list):
            if SkyCoord(qq['FG_RA']*u.deg,qq['FG_DEC']*u.deg).separation(zem_coord[zz]) < 5*u.arcsec:
                qq['Z_FG'] = zem['ZSYS_ZSYS'][0]
                qq['Z_FSIG'] = np.rint(zem['ZSYS_ERR'][0])
                qq['ZFG_LINE'] = zsys_line[np.min([zem['ZSYS_FLAG'][0],3])]   
    except AssertionError:
        pass
    try:
        assert 'J1433+0641' in qq['NAME'] # redshift is Null in the zIR structure 
        for zz,zem in enumerate(zem_list):
            if SkyCoord(qq['FG_RA']*u.deg,qq['FG_DEC']*u.deg).separation(zem_coord[zz]) < 5*u.arcsec:
                qq['Z_FG'] = zem['ZSYS_ZSYS'][0]
                qq['Z_FSIG'] = np.rint(zem['ZSYS_ERR'][0])
                qq['ZFG_LINE'] = zsys_line[np.min([zem['ZSYS_FLAG'][0],3])]
    except AssertionError:
        pass
len(QPQ9)

31

In [12]:
# Add QPQ7 pairs with NIR redshifts already measured 
c_QPQzIR = SkyCoord(ra=QPQ_zIR['FG_RA']*u.deg, dec=QPQ_zIR['FG_DEC']*u.deg)
c_QPQ7 = SkyCoord(ra=QPQ7['RAD']*u.deg, dec=QPQ7['DECD']*u.deg)
c_QPQ7_bg = SkyCoord(ra=QPQ7['RAD_BG']*u.deg,dec=QPQ7['DECD_BG']*u.deg)
kpc_amin = cosmo.kpc_proper_per_arcmin(QPQ7['Z_FG'])
ang_sep = c_QPQ7.separation(c_QPQ7_bg).to('arcmin')
                                          
for qq,cc in enumerate(c_QPQ7):
    # do not copy the entry with wrong spectral file 
    wrong_c = SkyCoord(ra=8.59753707028004*u.deg,dec=-10.832307685613417*u.deg)
    if c_QPQ7_bg[qq].separation(wrong_c).to('arcsec') < 0.5*u.arcsec:
        continue
    if len(np.where(cc.separation(c_QPQzIR).to('arcsec') < 0.5*u.arcsec)[0]) == 0:
        if kpc_amin[qq]*ang_sep[qq] < 300*u.kpc:
            if len(np.where(cc.separation(c_NIR) < 5*u.arcsec)[0]) > 0:
                print(cc.to_string('hmsdms'))
                for zz,zem in enumerate(zem_list):
                    if cc.separation(zem_coord[zz]) < 5*u.arcsec:
                        QPQ9.add_row({'FG_RA':QPQ7[qq]['RAD'],'FG_DEC':QPQ7[qq]['DECD'],'NAME':QPQ7[qq]['QSO_BG'],
                                      'RA':QPQ7[qq]['RAD_BG'],'DEC':QPQ7[qq]['DECD_BG'],
                                      'Z_FG':zem['ZSYS_ZSYS'][0],'Z_FSIG':np.rint(zem['ZSYS_ERR'][0]),
                                      'BG_Z':QPQ7[qq]['Z_BG'],'R_PHYS':QPQ7[qq]['R_PHYS'],
                                      'ZFG_LINE':zsys_line[np.min([zem['ZSYS_FLAG'][0],3])],
                                      'G_UV':QPQ7[qq]['G_UV']})
                        
QPQ9.sort(['FG_RA','FG_DEC'])

12h31m43.0985s +00d28m46.2929s


In [13]:
# Add QPQ7 pairs with MgII redshifts
c_QPQ9 = SkyCoord(ra=QPQ9['FG_RA']*u.deg, dec=QPQ9['FG_DEC']*u.deg)
for qq,cc in enumerate(c_QPQ7):
    # do not copy the entry with wrong spectral file 
    wrong_c = SkyCoord(ra=8.59753707028004*u.deg,dec=-10.832307685613417*u.deg)
    if c_QPQ7_bg[qq].separation(wrong_c).to('arcsec') < 0.5*u.arcsec:
        continue
    if len(np.where(cc.separation(c_QPQ9).to('arcsec') < 0.5*u.arcsec)[0]) == 0:
        if kpc_amin[qq]*ang_sep[qq] < 300*u.kpc:
            if QPQ7['Z_FSIG'][qq] < 300:
                QPQ9.add_row({'FG_RA':QPQ7['RAD'][qq],'FG_DEC':QPQ7['DECD'][qq],'NAME':QPQ7['QSO_BG'][qq],
                              'RA':QPQ7['RAD_BG'][qq],'DEC':QPQ7['DECD_BG'][qq],'Z_FG':QPQ7['Z_FG'][qq],'Z_FSIG':272,
                              'BG_Z':QPQ7['Z_BG'][qq],'R_PHYS':QPQ7['R_PHYS'][qq],'ZFG_LINE':'MgII',
                              'G_UV':QPQ7['G_UV'][qq]})

QPQ9.sort(['FG_RA','FG_DEC'])

In [14]:
# write
xxf.table_to_fits(QPQ9,'qpq9_final.fits')

# Check pairs in QPQ database and have NIR

In [15]:
# Find and add those pairs 
QPQ9 = xxf.bintab_to_table('qpq9_final.fits')
c_QPQ9 = SkyCoord(QPQ9['FG_RA']*u.deg,QPQ9['FG_DEC']*u.deg)
c_QPQ9_bg = SkyCoord(ra=QPQ9['RA']*u.deg,dec=QPQ9['DEC']*u.deg)
qpq_fil = '/Users/lwymarie/Documents/Databases/qpq_oir_spec.hdf5'
qpqsp = SpecDB(db_file=qpq_fil,verbose=False,idkey='QPQ_ID')
ID_fg, ID_bg = qpqsp.qcat.pairs(0.92*u.arcmin, 3000.*u.km/u.s)
c_qpqsp = SkyCoord(qpqsp.cat['RA'][ID_fg]*u.deg,qpqsp.cat['DEC'][ID_fg]*u.deg)
c_qpqsp_bg = SkyCoord(qpqsp.cat['RA'][ID_bg]*u.deg,qpqsp.cat['DEC'][ID_bg]*u.deg)
kpc_amin = cosmo.kpc_proper_per_arcmin(qpqsp.cat['zem'][ID_fg])
ang_sep = c_qpqsp.separation(c_qpqsp_bg).to('arcmin')
for qq,cc in enumerate(c_qpqsp):
    if kpc_amin[qq]*ang_sep[qq] < 300 *u.kpc: 
        if len(np.where(cc.separation(c_QPQ9) < 0.5*u.arcsec)[0]) == 0: # pair not in QPQ9 already
            if len(np.where(cc.separation(c_NIR) < 5*u.arcsec)[0]) > 0:
                print(cc.to_string('hmsdms'))
                #J0225 has no good line measured. Skip.
                if cc.separation(SkyCoord('02h25m17.68s +00d48m22s')) < 5*u.arcsec:
                    continue
                #J0239 add [OIII]. Did not pass Lyb cut. Add back. 
                #J1112 has good Hb, but currently it's MgII from QPQ7. 
                #The redshift from QPQ7 or Hb here does not pass the > 3000 km/s requirement. Skip. 
                if cc.separation(SkyCoord('11h12m42.69s +66d11m52.8s')) < 5*u.arcsec:
                    continue
                #J1215 add [OIII].
                #J2338 add [OIII]. 
                name = 'BOSSJ'
                name = name + c_qpqsp_bg[qq].to_string('hmsdms')[0:2] + c_qpqsp_bg[qq].to_string('hmsdms')[3:5]
                ipos = c_qpqsp_bg[qq].to_string('hmsdms').rfind(' ')+1
                name = name + c_qpqsp_bg[qq].to_string('hmsdms')[ipos:ipos+3] + c_qpqsp_bg[qq].to_string('hmsdms')[ipos+4:ipos+6]
                index = np.where(zem_coord.separation(cc) < 5*u.arcsec)[0]
                if len(index) == 0:
                    print('no match found for',cc.to_string('hmsdms'),c_qpqsp_bg[qq].to_string('hmsdms'))
                    continue
                QPQ9.add_row({'FG_RA':qpqsp.cat[ID_fg[qq]]['RA'],'FG_DEC':qpqsp.cat[ID_fg[qq]]['DEC'],'NAME':name,
                              'RA':qpqsp.cat[ID_bg[qq]]['RA'],'DEC':qpqsp.cat[ID_bg[qq]]['DEC'],
                              'Z_FG':zem_list[index[0]]['ZSYS_ZSYS'][0],'Z_FSIG':np.rint(zem_list[index[0]]['ZSYS_ERR'][0]),
                                      'BG_Z':qpqsp.cat[ID_bg[qq]]['zem'],'R_PHYS':(kpc_amin[qq]*ang_sep[qq]).value,
                                      'ZFG_LINE':zsys_line[np.min([zem_list[index[0]]['ZSYS_FLAG'][0],3])]})
QPQ9.sort(['FG_RA','FG_DEC'])

x.fits.bintab_to_table: Reading qpq9_final.fits
Database is QPQ
Created on 2017-Jan-31
08h14m19.58s +32d50m18.7s
02h25m17.68s +00d48m22s
02h39m46.43s -01d06m40.4s
11h12m42.69s +66d11m52.8s
12h15m59s +57d16m16.6s


In [16]:
# write
xxf.table_to_fits(QPQ9,'qpq9_final.fits')

# Find igmspec pairs with NIR spectra

In [17]:
# Find and add those pairs 
QPQ9 = xxf.bintab_to_table('qpq9_final.fits')
c_QPQ9 = SkyCoord(QPQ9['FG_RA']*u.deg,QPQ9['FG_DEC']*u.deg)
c_QPQ9_bg = SkyCoord(ra=QPQ9['RA']*u.deg,dec=QPQ9['DEC']*u.deg)
#igmsp = IgmSpec()
igmsp = IgmSpec(version="02.1",
                groups=['BOSS_DR12','COS-Dwarfs','COS-Halos',
                        'ESI_DLA','GGG','HD-LLS_DR1',
                        'HDLA100','HSTQSO','HST_z2',
                        'KODIAQ_DR1','SDSS_DR7','XQ-100',
                        'UVES_Dall','UVpSM4','MUSoDLA'])
ID_fg, ID_bg = igmsp.qcat.pairs(0.92*u.arcmin, 3000.*u.km/u.s) # at z = 0.4, 0.92' = 300 kpc
c_igmsp = SkyCoord(igmsp.cat['RA'][ID_fg]*u.deg,igmsp.cat['DEC'][ID_fg]*u.deg)
c_igmsp_bg = SkyCoord(igmsp.cat['RA'][ID_bg]*u.deg,igmsp.cat['DEC'][ID_bg]*u.deg)
kpc_amin = cosmo.kpc_proper_per_arcmin(igmsp.cat['zem'][ID_fg])
ang_sep = c_igmsp.separation(c_igmsp_bg).to('arcmin')

for qq,cc in enumerate(c_igmsp):
    if kpc_amin[qq]*ang_sep[qq] < 300 *u.kpc: 
        if len(np.where(cc.separation(c_QPQ9) < 0.5*u.arcsec)[0]) == 0: # pair not in QPQ9 already
            if len(np.where(cc.separation(c_NIR) < 5*u.arcsec)[0]) > 0:
                print(cc.to_string('hmsdms'))         
                # Skip J0225. No good line measured. 
                if cc.separation(SkyCoord('02h25m17.68s +00d48m22s')) < 5*u.arcsec:
                    continue
                # Skip J0201. Binary quasar, with wrong redshift in catalog. 
                if cc.separation(SkyCoord('02h01m43.4873s +00d32m22.713s')) < 5*u.arcsec:
                    continue
                name = 'SDSSJ'
                name = name + c_igmsp_bg[qq].to_string('hmsdms')[0:2] + c_igmsp_bg[qq].to_string('hmsdms')[3:5]
                ipos = c_igmsp_bg[qq].to_string('hmsdms').rfind(' ')+1
                name = name + c_igmsp_bg[qq].to_string('hmsdms')[ipos:ipos+3] + c_igmsp_bg[qq].to_string('hmsdms')[ipos+4:ipos+6]
                index = np.where(zem_coord.separation(cc) < 5*u.arcsec)[0]
                QPQ9.add_row({'FG_RA':igmsp.cat[ID_fg[qq]]['RA'],'FG_DEC':igmsp.cat[ID_fg[qq]]['DEC'],'NAME':name,
                              'RA':igmsp.cat[ID_bg[qq]]['RA'],'DEC':igmsp.cat[ID_bg[qq]]['DEC'],
                              'Z_FG':zem_list[index[0]]['ZSYS_ZSYS'][0],'Z_FSIG':np.rint(zem_list[index[0]]['ZSYS_ERR'][0]),
                                      'BG_Z':igmsp.cat[ID_bg[qq]]['zem'],'R_PHYS':(kpc_amin[qq]*ang_sep[qq]).value,
                                      'ZFG_LINE':zsys_line[np.min([zem_list[index[0]]['ZSYS_FLAG'][0],3])]})
QPQ9.sort(['FG_RA','FG_DEC'])

x.fits.bintab_to_table: Reading qpq9_final.fits
Loading igmspec from /Users/lwymarie/Documents/Databases/IGMspec_DB_v02.1.hdf5
Database is igmspec
Created on 2017-Sep-18
02h25m17.6775s +00d48m21.9398s
23h38m45.1943s -00d03m27.1218s
02h01m43.4873s +00d32m22.713s
01h11m50.07s +14d01m41.5s


In [18]:
# write
xxf.table_to_fits(QPQ9,'qpq9_final.fits')

# Add igmspec pairs with HW10 redshifts 

In [54]:
count = 0 
QPQ9 = xxf.bintab_to_table('qpq9_final.fits')
for qq,cc in enumerate(c_igmsp):
    if kpc_amin[qq]*ang_sep[qq] < 300 *u.kpc: 
        if len(np.where(cc.separation(c_QPQ9) < 0.5*u.arcsec)[0]) == 0: # pair not in QPQ9 already
            if len(np.where(cc.separation(c_NIR) < 0.5*u.arcsec)[0]) == 0:                 
                if 'SDSS-HW' in igmsp.cat[ID_fg[qq]]['flag_zem']: # good z
                    # Match Johnson+15 and Farina+ samples' redshifts, and MgII redshift or better 
                    if (igmsp.cat[ID_fg[qq]]['zem'] >= 0.4) & (igmsp.cat[ID_fg[qq]]['zem'] <= 2.1): 
                        # Lya forest cut for MgII stack 
                        if 2796.354*(1+igmsp.cat['zem'][ID_fg[qq]]) > (1215.6701+6.)*(1+igmsp.cat['zem'][ID_bg[qq]]):
                            #print(cc.to_string('hmsdms')) 
                            if np.mod(igmsp.cat['flag_group'][ID_fg[qq]],2) == 0:
                                name = 'BOSSJ'
                            else:
                                name = 'SDSSJ'  
                            name = name + c_igmsp_bg[qq].to_string('hmsdms')[0:2]
                            name = name + c_igmsp_bg[qq].to_string('hmsdms')[3:5]
                            ipos = c_igmsp_bg[qq].to_string('hmsdms').rfind(' ')+1
                            name = name + c_igmsp_bg[qq].to_string('hmsdms')[ipos:ipos+3] 
                            name = name + c_igmsp_bg[qq].to_string('hmsdms')[ipos+4:ipos+6]
                            QPQ9.add_row({'FG_RA':igmsp.cat['RA'][ID_fg[qq]],'FG_DEC':igmsp.cat['DEC'][ID_fg[qq]],
                                          'NAME':name,
                                          'RA':igmsp.cat['RA'][ID_bg[qq]],'DEC':igmsp.cat['DEC'][ID_bg[qq]],
                                          'Z_FG':igmsp.cat['zem'][ID_fg[qq]],'Z_FSIG':272,
                                          'BG_Z':igmsp.cat['zem'][ID_bg[qq]],
                                          'R_PHYS':(kpc_amin[qq]*ang_sep[qq]).value,'ZFG_LINE':'SDSS-HW'})
#                            print(igmsp.cat['IGM_ID'][ID_fg[qq]],ID_fg[qq],igmsp.cat['IGM_ID'][ID_bg[qq]],ID_bg[qq])
                else: 
                    if (igmsp.cat[ID_fg[qq]]['zem'] >= 1.6) & (igmsp.cat[ID_fg[qq]]['zem'] <= 2.1): # where MgII accessible
                        bg_coord = SkyCoord(igmsp.cat['RA'][ID_bg[qq]]*u.deg,igmsp.cat['DEC'][ID_bg[qq]]*u.deg)
                        specs,metas = igmsp.spectra_from_coord(bg_coord,tol=0.5*u.arcsec,verbose=False,
                                                               groups=['BOSS_DR12','COS-Dwarfs','COS-Halos',
                                                                       'ESI_DLA','GGG','HD-LLS_DR1',
                                                                       'HDLA100','HSTQSO','HST_z2',
                                                                       'KODIAQ_DR1','SDSS_DR7','XQ-100',
                                                                       'UVES_Dall','UVpSM4','MUSoDLA'])
                        if metas is None:
                            continue
                        gd = 0 
                        Aband = [7595,7680]
                        Bband = [6868,6926]
                        # CII good
                        if 1334.5323*(1+igmsp.cat['zem'][ID_fg[qq]]) > (1215.6701+15.)*(1+igmsp.cat['zem'][ID_bg[qq]]):
                            for meta in metas:
                                stck_wvmin = 1334.5323*(1+ltu.z_from_dv(3000*u.km/u.s,igmsp.cat['zem'][ID_fg[qq]]))
                                stck_wvmax = 1334.5323*(1+ltu.z_from_dv(3000*u.km/u.s,igmsp.cat['zem'][ID_fg[qq]]))
                                if ((stck_wvmin > meta['WV_MIN']) and (stck_wvmax < meta['WV_MAX'])):
                                    if (((stck_wvmin >= Aband[0]) and (stck_wvmin <= Aband[1]) or 
                                         (stck_wvmax >= Aband[0]) and (stck_wvmax <= Aband[1])) or 
                                        ((stck_wvmin >= Bband[0]) and (stck_wvmin <= Bband[1]) or 
                                         (stck_wvmax >= Bband[0]) and (stck_wvmax <= Bband[1]))):
                                        continue
                                    gd = 1
                        # CIV good 
                        if 1548.195*(1+igmsp.cat['zem'][ID_fg[qq]]) > (1215.6701+15.)*(1+igmsp.cat['zem'][ID_bg[qq]]):
                            for meta in metas: 
                                stck_wvmin = 1548.195*(1+ltu.z_from_dv(3000*u.km/u.s,igmsp.cat['zem'][ID_fg[qq]]))
                                stck_wvmax = 1548.195*(1+ltu.z_from_dv(3000*u.km/u.s,igmsp.cat['zem'][ID_fg[qq]]))
                                if ((stck_wvmin > meta['WV_MIN']) and (stck_wvmax < meta['WV_MAX'])):
                                    if (((stck_wvmin >= Aband[0]) and (stck_wvmin <= Aband[1]) or 
                                         (stck_wvmax >= Aband[0]) and (stck_wvmax <= Aband[1])) or 
                                        ((stck_wvmin >= Bband[0]) and (stck_wvmin <= Bband[1]) or 
                                         (stck_wvmax >= Bband[0]) and (stck_wvmax <= Bband[1]))):
                                        continue
                                    gd = 1
                        # MgII good
                        if 2796.354*(1+igmsp.cat['zem'][ID_fg[qq]]) > (1215.6701+15.)*(1+igmsp.cat['zem'][ID_bg[qq]]):
                            for meta in metas:
                                stck_wvmin = 2796.354*(1+ltu.z_from_dv(3000*u.km/u.s,igmsp.cat['zem'][ID_fg[qq]]))
                                stck_wvmax = 2796.354*(1+ltu.z_from_dv(3000*u.km/u.s,igmsp.cat['zem'][ID_fg[qq]]))
                                if ((stck_wvmin > meta['WV_MIN']) and (stck_wvmax < meta['WV_MAX'])):
                                    if (((stck_wvmin >= Aband[0]) and (stck_wvmin <= Aband[1]) or 
                                         (stck_wvmax >= Aband[0]) and (stck_wvmax <= Aband[1])) or 
                                        ((stck_wvmin >= Bband[0]) and (stck_wvmin <= Bband[1]) or 
                                         (stck_wvmax >= Bband[0]) and (stck_wvmax <= Bband[1]))):
                                        continue
                                    gd = 1
                        if gd == 1:
                            count +=1 
                            print(cc.to_string('hmsdms',),igmsp.cat['zem'][ID_fg[qq]],igmsp.cat['zem'][ID_bg[qq]]) 
QPQ9.sort(['FG_RA','FG_DEC'])
count

x.fits.bintab_to_table: Reading qpq9_final.fits
Staged 1 spectra totalling 9.3e-05 Gb
Loaded spectra
(u'00h02m54.7157s +21d39m06.0465s', 1.8773361261016361, 2.6916261150827276)
Staged 1 spectra totalling 9.3e-05 Gb
Loaded spectra
(u'00h09m56.5739s +26d16m28.7461s', 1.9968066267917532, 2.4392336788494138)
Staged 1 spectra totalling 9.3e-05 Gb
Loaded spectra
(u'00h10m10.1842s +32d59m51.1173s', 2.0434456426466707, 2.2232947651795292)
Staged 1 spectra totalling 9.3e-05 Gb
Loaded spectra
Staged 1 spectra totalling 8e-05 Gb
Loaded spectra
(u'00h42m20.6566s +00d32m18.6839s', 1.9263012534510169, 3.0480374551098768)
Staged 1 spectra totalling 9.3e-05 Gb
Loaded spectra
(u'00h47m45.4931s +31d01m20.2527s', 1.9700115532542901, 2.6954714993689)
Staged 1 spectra totalling 9.3e-05 Gb
Loaded spectra
Staged 1 spectra totalling 9.3e-05 Gb
Loaded spectra
(u'01h03m38.5234s +27d38m23.7954s', 1.9453210850068485, 2.1768046232607845)
Staged 1 spectra totalling 9.3e-05 Gb
Loaded spectra
(u'01h08m10.8853s +22d14

180

In [36]:
len(QPQ9),QPQ9['NAME','ZFG_LINE','Z_FG'][375:400]

(655, <QTable length=25>
      NAME      ZFG_LINE      Z_FG     
     str14       str36      float64    
 -------------- -------- --------------
 BOSSJ1225+5709  SDSS-HW  1.89710700512
 BOSSJ1227+3518  SDSS-HW  1.14139699936
 BOSSJ1227-0015  SDSS-HW 0.628700017929
 BOSSJ1229+3430  SDSS-HW  1.24029195309
 BOSSJ1231+3515  SDSS-HW  1.56726396084
 BOSSJ1231+0029   [OIII]  3.20117358704
 BOSSJ1232+5655  SDSS-HW 0.428507000208
 BOSSJ1233+1246  SDSS-HW  1.54952001572
 BOSSJ1235+3758  SDSS-HW  1.21141600609
 BOSSJ1235+4258  SDSS-HW  1.18074905872
            ...      ...            ...
 BOSSJ1239+3955  SDSS-HW  1.84168601036
 BOSSJ1241+4548  SDSS-HW 0.472916007042
 BOSSJ1242+4924  SDSS-HW 0.566568017006
 BOSSJ1242+5331  SDSS-HW  1.71729803085
 BOSSJ1242+2419  SDSS-HW  1.48176300526
 BOSSJ1242+1817  SDSS-HW 0.976599991322
 BOSSJ1243+5922  SDSS-HW  1.34902095795
 BOSSJ1244+3442  SDSS-HW   1.1418479681
 BOSSJ1244+3916  SDSS-HW  1.99733901024
 BOSSJ1244+3208  SDSS-HW  1.99100005627)

In [41]:
# write
xxf.table_to_fits(QPQ9,'qpq9_final.fits')

In [ ]:
# Find pairs in QPQ database that have [OIII] at accessible wavelengths, but not in QPQ9 
QPQ9 = xxf.bintab_to_table('qpq9_final.fits')
c_QPQ9 = SkyCoord(QPQ9['FG_RA']*u.deg,QPQ9['FG_DEC']*u.deg)
c_QPQ9_bg = SkyCoord(ra=QPQ9['RA']*u.deg,dec=QPQ9['DEC']*u.deg)
qpq_fil = '/Users/lwymarie/Documents/Databases/qpq_oir_spec.hdf5'
qpqsp = SpecDB(db_file=qpq_fil,verbose=False,idkey='QPQ_ID')
ID_fg, ID_bg = qpqsp.qcat.pairs(0.92*u.arcmin, 3000.*u.km/u.s)
c_qpqsp = SkyCoord(qpqsp.cat['RA'][ID_fg]*u.deg,qpqsp.cat['DEC'][ID_fg]*u.deg)
c_qpqsp_bg = SkyCoord(qpqsp.cat['RA'][ID_bg]*u.deg,qpqsp.cat['DEC'][ID_bg]*u.deg)
kpc_amin = cosmo.kpc_proper_per_arcmin(qpqsp.cat['zem'][ID_fg])
ang_sep = c_qpqsp.separation(c_qpqsp_bg).to('arcmin')
for qq,cc in enumerate(c_qpqsp):
    if kpc_amin[qq]*ang_sep[qq] < 300 *u.kpc: 
        if len(np.where(cc.separation(c_QPQ9) < 0.5*u.arcsec)[0]) == 0: # pair not in QPQ9 already
                if (((qpqsp.cat['zem'][ID_fg[qq]] > 1.6) & (qpqsp.cat['zem'][ID_fg[qq]] < 1.67)) | 
                    ((qpqsp.cat['zem'][ID_fg[qq]] > 1.99) & (qpqsp.cat['zem'][ID_fg[qq]] < 2.60)) | 
                    ((qpqsp.cat['zem'][ID_fg[qq]] > 3.08) & (qpqsp.cat['zem'][ID_fg[qq]] < 3.76))):
                    print(qpqsp.cat['zem'][ID_fg[qq]],cc.to_string('hmsdms'),c_qpqsp_bg[qq].to_string('hmsdms'))

In [ ]:
# Try 13h44m41.1s -00d49m50.2s with TSpec

In [ ]:
# Find pairs in igmspec database that have [OIII] at accessible wavelengths, but not in QPQ9 
igmsp = IgmSpec()
ID_fg, ID_bg = igmsp.qcat.pairs(0.92*u.arcmin, 3000.*u.km/u.s)
c_igmsp = SkyCoord(igmsp.cat['RA'][ID_fg]*u.deg,igmsp.cat['DEC'][ID_fg]*u.deg)
c_igmsp_bg = SkyCoord(igmsp.cat['RA'][ID_bg]*u.deg,igmsp.cat['DEC'][ID_bg]*u.deg)
kpc_amin = cosmo.kpc_proper_per_arcmin(igmsp.cat['zem'][ID_fg])
ang_sep = c_igmsp.separation(c_igmsp_bg).to('arcmin')
for qq,cc in enumerate(c_igmsp):
    if kpc_amin[qq]*ang_sep[qq] < 300 *u.kpc: 
        if len(np.where(cc.separation(c_QPQ9) < 0.5*u.arcsec)[0]) == 0: # pair not in QPQ9 already
                if (((igmsp.cat['zem'][ID_fg[qq]] > 1.6) & (igmsp.cat['zem'][ID_fg[qq]] < 1.67)) | 
                    ((igmsp.cat['zem'][ID_fg[qq]] > 1.99) & (igmsp.cat['zem'][ID_fg[qq]] < 2.60)) | 
                    ((igmsp.cat['zem'][ID_fg[qq]] > 3.08) & (igmsp.cat['zem'][ID_fg[qq]] < 3.76))):
                    print(igmsp.cat['zem'][ID_fg[qq]],cc.to_string('hmsdms'),c_igmsp_bg[qq].to_string('hmsdms'))

# Replace HW10 redshifts with MgII redshifts re-measured

In [42]:
# Load QPQ9
QPQ9 = xxf.bintab_to_table('qpq9_final.fits')
c_QPQ9 = SkyCoord(QPQ9['FG_RA']*u.deg,QPQ9['FG_DEC']*u.deg)
# Load the zem structures 
zem_list = []
zem_coord = []
zem_fil = glob.glob(os.getenv('QPQ9')+'Analysis/Redshifts/QPQ_on_HW/zem/*')
for ff in zem_fil:
    zem = Table.read(ff)
    zem_list.append(zem)
    ipos = zem['SPEC_FIL'][0].rfind('/SDSSJ')+6
    radec = zem['SPEC_FIL'][0][ipos:]
    ipos2 = radec.rfind('+')
    if ipos2 == -1:
        ipos2 = radec.rfind('-')
    ipos3 = radec.rfind('_F.fits')
    if ipos3 == -1:
        ipos3 = radec.rfind('.fits')
    RA = radec[0:2]+'h'+radec[2:4]+'m'+radec[4:ipos2]+'s'
    dec = radec[ipos2:ipos2+3]+'d'+radec[ipos2+3:ipos2+5]+'m'+radec[ipos2+5:ipos3]+'s'
    zem_coord.append(SkyCoord(RA,dec))
zem_coord = SkyCoord(zem_coord)
# Match coordinates and replace redshifts
for qq,cc in enumerate(c_QPQ9):
    match = np.where(SkyCoord(cc).separation(zem_coord) < 5*u.arcsec)[0]
    if len(match) > 1:
        pdb.set_trace()
    if (len(match) == 1) and (zem_list[match[0]]['MGII_MODE'][0] != -1):
        QPQ9[qq]['Z_FG'] = zem_list[match[0]]['ZSYS_ZSYS'][0]
        QPQ9[qq]['ZFG_LINE'] = 'MgII'
        print(SkyCoord(cc).to_string('hmsdms'),'replaced')
# Remove z > 1.6 quasars with no good MgII
sv_qq = []
for qq,cc in enumerate(c_QPQ9): 
    if (cc.separation(SkyCoord('10h24m43.99s +26d16m15.1s')) < 5*u.arcsec): 
        sv_qq.append(qq)
    if (QPQ9[qq]['Z_FG'] > 1.6) and (QPQ9[qq]['ZFG_LINE'] == 'SDSS-HW'):
        sv_qq.append(qq)
print('Removing',QPQ9[sv_qq]['NAME'])
QPQ9.remove_rows(sv_qq)

x.fits.bintab_to_table: Reading qpq9_final.fits
(u'00h47m57.2575s +14d47m41.9604s', 'replaced')
(u'01h03m23.8409s -00d02m54.204s', 'replaced')
(u'02h14m16.9579s -00d52m29.1468s', 'replaced')
(u'02h24m47.8961s -00d47m00.3552s', 'replaced')
(u'02h50m38.6916s -00d47m39.1056s', 'replaced')
(u'08h09m45.1728s +45d39m18.108s', 'replaced')
(u'08h12m23.1689s +26d20m00.906s', 'replaced')
(u'08h18m32.8654s +12d32m19.9032s', 'replaced')
(u'08h26m42.8635s +47d08m29.5296s', 'replaced')
(u'08h30m30.3792s +54d52m28.7724s', 'replaced')
(u'08h38m54.5237s +46d21m24.3936s', 'replaced')
(u'08h56m29.4814s +55d14m50.244s', 'replaced')
(u'08h57m37.5814s +39d01m20.4636s', 'replaced')
(u'09h04m17.9402s +00d41m48.2496s', 'replaced')
(u'09h06m57.7973s +10d01m21.4356s', 'replaced')
(u'09h12m17.5714s +41d39m33.516s', 'replaced')
(u'09h12m34.273s +30d56m16.2168s', 'replaced')
(u'09h15m51.7238s +01d19m00.1776s', 'replaced')
(u'09h33m17.4271s +59d20m27.4488s', 'replaced')
(u'09h41m33.642s +23d08m40.0524s', 'replaced')

In [5]:
len(QPQ9),QPQ9['ZFG_LINE','NAME'][210:230]

(573, <QTable length=20>
 ZFG_LINE      NAME     
  str36       str14     
 -------- --------------
  SDSS-HW BOSSJ1017+4740
     MgII BOSSJ1017+6226
  SDSS-HW BOSSJ1018+2913
     MgII BOSSJ1020+6119
  SDSS-HW BOSSJ1020+4323
  SDSS-HW BOSSJ1022+2925
  SDSS-HW BOSSJ1022+2945
  SDSS-HW BOSSJ1023+1220
  SDSS-HW BOSSJ1024+3222
  SDSS-HW BOSSJ1024+4051
  SDSS-HW BOSSJ1025+3156
 H$\beta$  APOJ1026+4614
  SDSS-HW BOSSJ1032+1850
     MgII BOSSJ1034+0856
  SDSS-HW BOSSJ1035+2250
     MgII BOSSJ1036+5012
  SDSS-HW BOSSJ1037+4401
  SDSS-HW BOSSJ1037+0736
   [OIII] BOSSJ1039+5026
  SDSS-HW BOSSJ1039+6336)

In [43]:
## write
xxf.table_to_fits(QPQ9,'qpq9_final.fits')

# Add and update g_UV values 

In [4]:
# Search SDSS DR12 catalog for magnitudes 
QPQ9 = xxf.bintab_to_table('qpq9_final.fits')
c_QPQ9 = SkyCoord(QPQ9['FG_RA']*u.deg,QPQ9['FG_DEC']*u.deg)
c_QPQ9_bg = SkyCoord(ra=QPQ9['RA']*u.deg,dec=QPQ9['DEC']*u.deg)
hdulist = fits.open('/Volumes/Data/Data of surveys/SDSS/DR12Q.fits')
SDSSdr12 = hdulist[1].data
c_SDSSdr12 = SkyCoord(list(SDSSdr12.field('RA'))*u.deg,list(SDSSdr12.field('DEC'))*u.deg)
hdulist = fits.open('/Volumes/Data/Data of surveys/SDSS/dr7qso.fit')
SDSSdr7 = hdulist[1].data
c_SDSSdr7 = SkyCoord(list(SDSSdr7.field('RA'))*u.deg,list(SDSSdr7.field('DEC'))*u.deg)

x.fits.bintab_to_table: Reading qpq9_final.fits


In [6]:
# Put in g_UV by hand for a quasar that does not have SDSS spectra but photometry and no existing g_UV values 
for qq,cc in enumerate(c_QPQ9):
    if QPQ9[qq]['G_UV'] == 0.:
        if len(np.where(cc.separation(c_SDSSdr12) < 0.5*u.arcsec)[0]) == 0:
            if len(np.where(cc.separation(c_SDSSdr7) < 0.5*u.arcsec)[0]) == 0:
                ang_sep = cc.separation(c_QPQ9_bg[qq]).to('arcsec')
                print(cc.to_string('hmsdms'),QPQ9[qq]['Z_FG'],ang_sep,qq)
                QPQ9[qq]['G_UV'] = 4624

(u'01h11m50.07s +14d01m41.5s', 2.4851422501937015, <Angle 35.64055373774893 arcsec>, 27)


In [7]:
# Write a file for IDL input, for rescaling QPQ g_UV values to F.-G. Lya forest flux 
gUV_exist = np.where(QPQ9['G_UV'] != 0.)[0]
data = Table([QPQ9[gUV_exist]['FG_RA'],QPQ9[gUV_exist]['FG_DEC'],QPQ9[gUV_exist]['Z_FG'],QPQ9[gUV_exist]['G_UV']],
             names=['FG_RA','FG_DEC','Z_FG','OLD_GUV'])
ascii.write(data,'QPQ_gUV_idl_input.txt')

# Write a file for IDL input, for calculating new g_UV values 
gUV_none = np.where(QPQ9['G_UV'] == 0.)[0]
ang_sep = []
SDSSfilter = []
mag = []
filter_beg = [3125.0, 3880.0, 5480.0, 6790.0, 8090.0]
filter_names = ['u','g','r','i','z']
for qq,cc in enumerate(c_QPQ9[gUV_none]):
    ang_sep.append(cc.separation(c_QPQ9_bg[gUV_none[qq]]).to('arcsec').value)
    min_beg = 1215.6701*(1+QPQ9[gUV_none[qq]]['Z_FG'])
    good_inds = np.where(filter_beg >= min_beg)
    SDSSfilter.append(filter_names[np.min(good_inds)])
    in_dr12 = np.where(cc.separation(c_SDSSdr12) < 0.5*u.arcsec)[0]
    if len(in_dr12) == 1:
        mag.append((SDSSdr12[in_dr12].field('PSFMAG')[0][good_inds]-
                    SDSSdr12[in_dr12].field('EXTINCTION')[0][good_inds])[0])
    else:
        in_dr7 = np.where(cc.separation(c_SDSSdr7) < 0.5*u.arcsec)[0]
        psfmag = [SDSSdr7[in_dr7].field('UMAG'),SDSSdr7[in_dr7].field('GMAG'),SDSSdr7[in_dr7].field('RMAG'),
                  SDSSdr7[in_dr7].field('IMAG'),SDSSdr7[in_dr7].field('ZMAG')]
        extinct = SDSSdr7[in_dr7].field('AU')*np.array([1,0.736,0.534,0.405,0.287])
        mag.append((psfmag[np.min(good_inds)]-extinct[np.min(good_inds)])[0])
data = Table([QPQ9[gUV_none]['FG_RA'],QPQ9[gUV_none]['FG_DEC'],QPQ9[gUV_none]['Z_FG'],ang_sep,SDSSfilter,mag],
             names=['FG_RA','FG_DEC','Z_FG','theta','filter','mag'])
ascii.write(data,'no_gUV_idl_input.txt')

In [8]:
# Read IDL output for rescaled QPQ g_UV values 
data = ascii.read('QPQ_gUV_idl_output.txt')
c_idl = SkyCoord(data['FG_RA']*u.deg,data['FG_DEC']*u.deg)
for qq,cc in enumerate(c_QPQ9):
    index = np.where(cc.separation(c_idl) < 0.5*u.arcsec)[0]
    if len(index) == 1:
        QPQ9[qq]['G_UV'] = data[index]['NEW_GUV']
# Read IDL output for new g_UV values
data = ascii.read('no_gUV_idl_output.txt')
c_idl = SkyCoord(data['FG_RA']*u.deg,data['FG_DEC']*u.deg)
for qq,cc in enumerate(c_QPQ9):
    index = np.where(cc.separation(c_idl) < 0.5*u.arcsec)[0]
    if len(index) == 1:
        QPQ9[qq]['G_UV'] = data[index]['GUV']

In [9]:
# write
xxf.table_to_fits(QPQ9,'qpq9_final.fits')

# Update systematic offsets and scatters of emission lines to Shen+16 

In [10]:
# Add the systematic biases, intrinsic scatter + measurement scatter
QPQ9 = xxf.bintab_to_table('qpq9_final.fits')
for ii,qq in enumerate(QPQ9):
    if qq['ZFG_LINE'] == 'MgII':
        # bring to z_em 
        qq['Z_FG'] = ltu.z_from_dv(+70.*u.km/u.s,qq['Z_FG'])
        # bring to z_sys
        qq['Z_FG'] = ltu.z_from_dv(+57.*u.km/u.s,qq['Z_FG'])
        # scatter
        qq['Z_FSIG'] = 226
    if qq['ZFG_LINE'] == 'H$\\beta$':
        # bring to z_sys
        qq['Z_FG'] = ltu.z_from_dv(+109.*u.km/u.s,qq['Z_FG'])
        # scatter 
        qq['Z_FSIG'] = 417
    if qq['ZFG_LINE'] == '[OIII]':
        # bring to z_em 
        qq['Z_FG'] = ltu.z_from_dv(-27.*u.km/u.s,qq['Z_FG'])
        # bring to z_sys
        qq['Z_FG'] = ltu.z_from_dv(+48.*u.km/u.s,qq['Z_FG'])
        # scatter
        qq['Z_FSIG'] = 68
    if qq['ZFG_LINE'] == 'H$\\alpha$':
        # scatter
        qq['Z_FSIG'] = 300
    if qq['ZFG_LINE'] == 'SDSS-HW':
        if qq['Z_FG'] >= 0.84:
            # MgII redshift, bring to z_sys
            qq['Z_FG'] = ltu.z_from_dv(+57.*u.km/u.s,qq['Z_FG'])
            # scatter
            qq['Z_FSIG'] = 226
        else:
            # [OIII] redshift, bring to zsys
            qq['Z_FG'] = ltu.z_from_dv(+48.*u.km/u.s,qq['Z_FG'])
            # scatter
            qq['Z_FSIG'] = 68

x.fits.bintab_to_table: Reading qpq9_final.fits


In [11]:
# write
xxf.table_to_fits(QPQ9,'qpq9_final.fits')

# copy to enigma

In [14]:
!cp qpq9_final.fits /Users/lwymarie/python/enigma/enigma/data/qpq/qpq9_final.fits
len(QPQ9)

573

In [12]:
QPQ9 = xxf.bintab_to_table('qpq9_final.fits')

x.fits.bintab_to_table: Reading qpq9_final.fits


In [13]:
len(QPQ9),QPQ9['Z_FG','ZFG_LINE','NAME'][320:340]

(573, <QTable length=20>
      Z_FG      ZFG_LINE      NAME     
    float64      str36       str14     
 -------------- -------- --------------
 0.803720759758  SDSS-HW BOSSJ1215+6538
 0.471676613109  SDSS-HW BOSSJ1215+1954
  1.11163241918  SDSS-HW BOSSJ1215+3126
    1.929984981   [OIII] BOSSJ1215+5715
 0.619317262336  SDSS-HW BOSSJ1218+5739
 0.790264645043  SDSS-HW BOSSJ1219+5801
 0.585753877852  SDSS-HW BOSSJ1224+3721
 0.499351039751  SDSS-HW BOSSJ1224+4307
  1.89531880539     MgII BOSSJ1225+5709
  1.14180418517  SDSS-HW BOSSJ1227+3518
 0.628960811216  SDSS-HW BOSSJ1227-0015
  1.24071794373  SDSS-HW BOSSJ1229+3430
  1.56775212509  SDSS-HW BOSSJ1231+3515
  3.20146788309   [OIII] BOSSJ1231+0029
 0.428735737871  SDSS-HW BOSSJ1232+5655
  1.55000480596  SDSS-HW BOSSJ1233+1246
  1.21183650599  SDSS-HW BOSSJ1235+3758
  1.18116372731  SDSS-HW BOSSJ1235+4258
 0.558830560183  SDSS-HW BOSSJ1235+6451
  1.41772768038  SDSS-HW BOSSJ1238+4634)

In [5]:
from enigma.qpq.qpq import QPQ

Reading /Users/lwymarie/python/enigma/enigma/data/qpq/qpq9_final.fits


In [13]:
qpq9 = QPQ('QPQ9')
qpq9.cull(1334.5323*u.AA,4)
len(qpq9.data)
qpq9.cull(1334.5323*u.AA,4,vsig_cut=400*u.km/u.s)
len(qpq9.data)

Reading /Users/lwymarie/python/enigma/enigma/data/qpq/qpq9_final.fits


63

In [35]:
for qq in QPQ9:
    if (1.6 < qq['Z_FG'] < 2.1) and (qq['ZFG_LINE'] == 'MgII'):
        print(SkyCoord(qq['FG_RA']*u.deg,qq['FG_DEC']*u.deg).to_string('hmsdms'),qq['Z_FG'])
igmsp = IgmSpec()
ID_fg, ID_bg = igmsp.qcat.pairs(0.92*u.arcmin, 3000.*u.km/u.s) # at z = 0.4, 0.92' = 300 kpc
c_igmsp = SkyCoord(igmsp.cat['RA'][ID_fg]*u.deg,igmsp.cat['DEC'][ID_fg]*u.deg)
c_igmsp_bg = SkyCoord(igmsp.cat['RA'][ID_bg]*u.deg,igmsp.cat['DEC'][ID_bg]*u.deg)
kpc_amin = cosmo.kpc_proper_per_arcmin(igmsp.cat['zem'][ID_fg])
ang_sep = c_igmsp.separation(c_igmsp_bg).to('arcmin')

(u'01h49m17.1049s -00d21m41.6461s', 1.6833870125798169)
(u'04h09m55.872s -04d11m26.9365s', 1.7166496442640746)
(u'07h52m59.8146s +40d11m28.2199s', 1.8844190841086819)
(u'08h28m43.368s +45d45m17.2955s', 1.8730148656904626)
(u'08h37m13.5609s +36d30m37.3449s', 1.8363777779524164)
(u'09h24m17.6496s +39d29m20.2547s', 1.8864405377263427)
(u'10h05m09.5996s +50d19m29.7953s', 1.8176172572629186)
(u'10h09m41.3471s +25d01m04.0768s', 1.8702530472315959)
(u'10h41m43.0992s +19d57m22.1954s', 1.8477168805195892)
(u'11h45m46.54s +03d22m36.7476s', 1.7663892099787759)
(u'13h27m40.582s +34d23m23.7427s', 1.6495955829403552)
(u'13h58m49.7126s +27d38m06.8626s', 1.9008334067438883)
(u'14h13m37.1791s +27d15m17.0969s', 1.6904535135359973)
(u'14h31m09.6746s +57d27m28.9987s', 1.6801735572566385)
(u'14h42m32.9167s +01d37m30.4042s', 1.8079358659220282)
(u'15h08m14.0616s +36d35m29.357s', 1.8493088795958224)
(u'15h33m28.8312s +14d25m42.5012s', 2.0781896649076277)
(u'15h59m47.7342s +49d43m07.2919s', 1.8614915067315916

In [100]:
overlap = np.where(c_igmsp.separation(SkyCoord('15h59m47.7342s +49d43m07.2919s'))
                                      < 5*u.arcsec)[0]
overlap,c_igmsp[overlap]
#print(ltu.z_from_dv(+57.*u.km/u.s,igmsp.cat['zem'][ID_fg[overlap]][0]))

(array([], dtype=int64), <SkyCoord (ICRS): (ra, dec) in deg
     []>)

In [20]:
count = 0 
for qq in QPQ9:
    if (qq['ZFG_LINE'] == 'SDSS-HW') and (1.6 < qq['Z_FG']):
        print(SkyCoord(qq['FG_RA']*u.deg,qq['FG_DEC']*u.deg).to_string('hmsdms'),qq['Z_FG'],qq['BG_Z'])
        count += 1
count

0

In [7]:
2800*(1+1.6002)-1215*(1+2.6073)

2897.6905000000006